In [180]:
import os.path
from googleapiclient.discovery import build
from google.oauth2 import service_account
import pandas as pd


SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
BASE_DIR = os.path.dirname(os.path.abspath("scripts.py"))
SERVICE_ACCOUNT_FILE = os.path.join(BASE_DIR, 'local-talent-364913-0febf38e0456.json')

credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# The ID and range of a sample spreadsheet.

SAMPLE_SPREADSHEET_ID = '1oYSVwlNr2NZSB6i2pNHyKN0aW34Y50jldQgEKHUJiVw'
SAMPLE_RANGE_NAME = 'Оценки'
SAMPLE_RANGE_NAME_2 = 'Логи'


service = build('sheets', 'v4', credentials=credentials)

sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME).execute()["values"][2:]
reasons = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range="Причины").execute()["values"][2:]
past_logs = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range="Логи").execute()["values"][2:]
accounts_1 = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range="Аккаунты").execute()["values"][2:]
accounts = [i[:4] for i in accounts_1]

In [181]:
# Fill names on each row

j = ''
for i in result:
    if i[0] == '':
        i[0] = j
    j = i [0]

In [182]:
df = pd.DataFrame(result)
df = df[df[1] != '']
df["sum_of_pluses"] = df[range(2,12)].T.apply(lambda x: x.str.count("\+")).sum()
df.drop(range(2,12), axis=1, inplace=True)
df.rename({0: 'student', 1: "reason"}, axis=1, inplace=True)


In [183]:
reasons_df = pd.DataFrame(reasons).drop([2, 3, 4], axis=1)
reasons_df.drop(reasons_df[reasons_df[1] == ""].index, inplace=True)

In [184]:
reasons_df.rename(columns={0: "reason", 1: "amount"}, inplace=True)
reasons_df.amount = reasons_df.amount.astype(int)

In [185]:
df = pd.merge(df, reasons_df)

In [186]:
df["to_add"] = df.sum_of_pluses * df["amount"].astype(int)

In [187]:
logs_df = pd.DataFrame(columns=['student', 'reason', 'amount', "date"])

In [188]:
from datetime import datetime
for row in df[df.sum_of_pluses != 0].values:
    for i in range(row[2]): 
        logs_df = logs_df.append({"student": row[0], "reason": row[1], "amount": row[3], "date": str(datetime.today()).split(".")[0]}, ignore_index=True)
logs_df.reset_index(drop=True, inplace=True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3396\4031595071.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  logs_df = logs_df.append({"student": row[0], "reason": row[1], "amount": row[3], "date": str(datetime.today()).split(".")[0]}, ignore_index=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3396\4031595071.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  logs_df = logs_df.append({"student": row[0], "reason": row[1], "amount": row[3], "date": str(datetime.today()).split(".")[0]}, ignore_index=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3396\4031595071.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  logs_df = logs_df.append({"student": row[0], "reason": row[1], "amount": row[3], 

In [189]:
if past_logs != []:
    past_logs_df = pd.DataFrame(past_logs)
    past_logs_df.rename(columns={0: 'student', 1: 'reason', 2: 'amount', 3: 'date'}, inplace=True)
    past_logs_df.amount = past_logs_df.amount.astype(int)
else:
    past_logs_df = pd.DataFrame(columns=['student', 'reason', 'amount', 'date'])

In [190]:
groups_past_logs = past_logs_df.groupby([past_logs_df.student, past_logs_df.reason, logs_df.amount], as_index=False).date.count()

In [191]:
groups_logs = logs_df.groupby([logs_df.student, logs_df.reason, logs_df.amount], as_index=False).date.count()

In [192]:
groups_logs = groups_logs.merge(groups_past_logs, on=['student', 'reason'], how="outer").fillna(0)

In [193]:
groups_logs["diff_count"] = groups_logs.date_x - groups_logs.date_y

In [194]:
groups_logs[groups_logs.diff_count != 0]

,student,reason,amount,date_x,date_y,diff_count
6,Лахтин Владимир,Участие в консультации (зеленая карточка),20,1,0.0,1.0
7,Лахтин Владимир,Участие в мероприятиях общих (просто участие ж...,20,1,0.0,1.0
8,Лахтин Владимир,Участие в метаворкшопе или спецворкшопе (зелен...,20,1,0.0,1.0


In [195]:
logs_to_add = pd.DataFrame(columns=['student', 'reason', 'amount', 'date'])
for event in groups_logs[groups_logs.diff_count != 0].values:
    df = pd.DataFrame([[event[0], event[1], event[2], str(datetime.today()).split(".")[0]]], columns=['student', 'reason', 'amount', 'date'])
    for _ in range(int(event[-1])):
        logs_to_add = pd.concat([logs_to_add, df], ignore_index=True)

# Вносим данные

In [196]:
l = []
for i in logs_to_add.values:
    l.append(list(i))

In [197]:
start_row = len(past_logs) + 3

In [198]:
results_2 = service.spreadsheets().values().batchUpdate(spreadsheetId=SAMPLE_SPREADSHEET_ID, body={
    "valueInputOption": "USER_ENTERED",
    "data": [
        {"range": f"{SAMPLE_RANGE_NAME_2}!A{start_row}:D100",
         "majorDimension": "ROWS",    
         "values": l}
    ]
}).execute()